In [4]:
#!pip install plotly
# jupyter nbconvert Coach_Impact_Analysis.ipynb --to slides  --no-input 
#! .\getvereinsdb.cmd
#!pip install plotly

In [5]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import plotly.express as px

def query_database(query,database='verbands.db'):
    '''
    Load data from database
    :param query: Specifies in an SQL Select statement which data to load
    :param database: Path to database
    :return: Pandas Dataframe of resultset
    '''
    cnx = sqlite3.connect(database)
    df = pd.read_sql_query(query, cnx)
    cnx.commit()
    cnx.close()
    return df

In [6]:
## Queries
query_leagues = """
select league,season, count(*) as games, count(distinct(t.name)) as teams
from teams t join  (
	select substr(result,0,instr(result, ':')) as homeresult,
		substr(result,instr(result, ':') + 1) as awayresult, cast(substr(round,0,instr(round, '.')) as INT) as current_round,
		CASE 
		   WHEN replace(date,'-','') between '20110701' and '20120701' THEN '2011/12'
		   WHEN replace(date,'-','') between '20120701' and '20130701' THEN '2012/13'
		   WHEN replace(date,'-','') between '20130701' and '20140701' THEN '2013/14'
		   WHEN replace(date,'-','') between '20140701' and '20150701' THEN '2014/15'
		   WHEN replace(date,'-','') between '20150701' and '20160701' THEN '2015/16'
		   WHEN replace(date,'-','') between '20160701' and '20170701' THEN '2016/17'
		   WHEN replace(date,'-','') between '20170701' and '20180701' THEN '2017/18'
		   WHEN replace(date,'-','') between '20180701' and '20190701' THEN '2018/19'
		   WHEN replace(date,'-','') between '20190701' and '20200701' THEN '2019/20'
		   WHEN replace(date,'-','') between '20200701' and '20210701' THEN '2020/21'
		   WHEN replace(date,'-','') between '20210701' and '20220701' THEN '2021/22'
		   WHEN replace(date,'-','') between '20220701' and '20230701' THEN '2022/23'
		   ELSE 'undefined'
		END as season ,* from matches where current_round != 0 ) 
		m on (t.id=m.away_team_id or t.id=m.home_team_id) 
        where replace(date,'-','') between '{}' and '{}' and league like "{}"
group by league,season
order by league,season desc
"""


query_spiele_spieler = """
Select * from 
(
select substr(result,0,instr(result, ':')) as homeresult,
		substr(result,instr(result, ':') + 1) as awayresult, cast(substr(round,0,instr(round, '.')) as INT) as current_round,
		CASE 
		   WHEN replace(date,'-','') between '20110701' and '20120701' THEN '2011/12'
		   WHEN replace(date,'-','') between '20120701' and '20130701' THEN '2012/13'
		   WHEN replace(date,'-','') between '20130701' and '20140701' THEN '2013/14'
		   WHEN replace(date,'-','') between '20140701' and '20150701' THEN '2014/15'
		   WHEN replace(date,'-','') between '20150701' and '20160701' THEN '2015/16'
		   WHEN replace(date,'-','') between '20160701' and '20170701' THEN '2016/17'
		   WHEN replace(date,'-','') between '20170701' and '20180701' THEN '2017/18'
		   WHEN replace(date,'-','') between '20180701' and '20190701' THEN '2018/19'
		   WHEN replace(date,'-','') between '20190701' and '20200701' THEN '2019/20'
		   WHEN replace(date,'-','') between '20200701' and '20210701' THEN '2020/21'
		   WHEN replace(date,'-','') between '20210701' and '20220701' THEN '2021/22'
		   WHEN replace(date,'-','') between '20220701' and '20230701' THEN '2022/23'
		   ELSE 'undefined'
		END as season
		,* from matches
		where current_round != 0 ) m  
		join matches_players mp on m.id=mp.match_id 
		join players p on mp.player_id=p.id
		where p.name like 'Thomas Dragschitz' and result not like '%-%'
"""


query_team_leagues = """
Select t.name, m.league, count(distinct(season)) as seasons from 
(
select substr(result,0,instr(result, ':')) as homeresult,
		substr(result,instr(result, ':') + 1) as awayresult, cast(substr(round,0,instr(round, '.')) as INT) as current_round,
		CASE 
		   WHEN replace(date,'-','') between '20110701' and '20120701' THEN '2011/12'
		   WHEN replace(date,'-','') between '20120701' and '20130701' THEN '2012/13'
		   WHEN replace(date,'-','') between '20130701' and '20140701' THEN '2013/14'
		   WHEN replace(date,'-','') between '20140701' and '20150701' THEN '2014/15'
		   WHEN replace(date,'-','') between '20150701' and '20160701' THEN '2015/16'
		   WHEN replace(date,'-','') between '20160701' and '20170701' THEN '2016/17'
		   WHEN replace(date,'-','') between '20170701' and '20180701' THEN '2017/18'
		   WHEN replace(date,'-','') between '20180701' and '20190701' THEN '2018/19'
		   WHEN replace(date,'-','') between '20190701' and '20200701' THEN '2019/20'
		   WHEN replace(date,'-','') between '20200701' and '20210701' THEN '2020/21'
		   WHEN replace(date,'-','') between '20210701' and '20220701' THEN '2021/22'
		   WHEN replace(date,'-','') between '20220701' and '20230701' THEN '2022/23'
		   ELSE 'undefined'
		END as season
		,* from matches
		where current_round != 0 ) m  join teams t on (t.id=m.away_team_id or t.id=m.home_team_id)
where m.league not like '%Reserve%'
group by t.name, m.league
order by t.name, m.league
"""

# Spielerprofil

In [7]:
name = 'Thomas Dragschitz'
player=query_database(query="Select * from players where name like 'Thomas Dragschitz'")
print(player)

     id                                                url               name  \
0  3800  https://www.oefb.at/Profile/Spieler/654332?Tho...  Thomas Dragschitz   

  nationality birthyear  
0        None      None  


In [8]:
df=query_database(query_team_leagues,database="verbands.db")#.groupby("league").count().reset_index().iloc[:,:2].head(500)
df.sort_values("seasons",ascending=False)
df.groupby("name").count().reset_index().iloc[:,:2].sort_values("league",ascending=False)
#df[df["name"]=="Leithaprodersdorf"]
df[df["name"]=="FK Austria Wien"]

,name,league,seasons
83,FK Austria Wien,ADMIRAL Bundesliga - Grunddurchgang,1
84,FK Austria Wien,Coca-Cola Cup Bundesfinale 2016 - Finale,1
85,FK Austria Wien,Coca-Cola Cup Bundesfinale 2018 - Finale,1
86,FK Austria Wien,Planet Pure Frauen Bundesliga,1
87,FK Austria Wien,Tipico Bundesliga - Grunddurchgang,1
88,FK Austria Wien,tipp3-Bundesliga,1
89,FK Austria Wien,tipp3-Bundesliga powered by T-Mobile,1


In [9]:
query_database("Select count(*) from matches") #10.11.2022 - 11579
query_database("Select * from matches order by id desc limit 1") #10.11.2022 - 11309

,id,url,date,starttime,league,round,result,away_team_id,away_team_trainer_id,home_team_id,home_team_trainer_id
0,13131,https://www.oefb.at/bewerbe/Spiel/Aufstellung/...,2021-11-06,12:00,II. Liga Süd Reserve,15. Runde,3:0,180,534,91,262


In [10]:
# Anzahl der Spiele die in der Regionalliga während seiner Trainerzeit
from pyparsing import col


pd.set_option('display.max_rows', None)
query_database(query_leagues.format(0,9999999,"%"),database="verbands.db")#.groupby("league").count().reset_index().iloc[:,:2].head(500)

,league,season,games,teams
0,1. Kl. Mitte Cup - 1. Kl. Cup A,2022/23,2,2
1,1. Kl. Mitte Cup - 1. Kl. Cup D,2022/23,2,2
2,1. Klasse Mitte,2022/23,156,13
3,1. Klasse Mitte,2021/22,366,14
4,1. Klasse Mitte,2018/19,366,14
5,1. Klasse Mitte Reserve,2022/23,132,12
6,1. Klasse Mitte Reserve,2021/22,312,13
7,1. Klasse Mitte Reserve,2020/21,92,13
8,1. Klasse Mitte Reserve,2018/19,312,13
9,1. Klasse Nord,undefined,2,2


In [11]:
query_teams_end_of_season=""" 
select name,league,season,points,url
from (
select 
sum(
	CASE WHEN homeresult==awayresult THEN 1 WHEN homeresult>awayresult and m.home_team_id==t.id THEN 3 WHEN homeresult<awayresult and m.away_team_id==t.id THEN 3 ELSE 0 END
    ) points,
	sum(
	CASE WHEN m.home_team_id==t.id THEN homeresult WHEN  m.away_team_id==t.id THEN awayresult ELSE 0 END
	)  scored,
    sum(
	CASE WHEN m.home_team_id==t.id THEN awayresult WHEN  m.away_team_id==t.id THEN homeresult  ELSE 0 END
	)  conceded,
	 *
from teams t join  (
select substr(result,0,instr(result, ':')) as homeresult,
		substr(result,instr(result, ':') + 1) as awayresult,
		CASE 
		   WHEN replace(date,'-','') between '20110701' and '20120701' THEN '2011/12'
		   WHEN replace(date,'-','') between '20120701' and '20130701' THEN '2012/13'
		   WHEN replace(date,'-','') between '20130701' and '20140701' THEN '2013/14'
		   WHEN replace(date,'-','') between '20140701' and '20150701' THEN '2014/15'
		   WHEN replace(date,'-','') between '20150701' and '20160701' THEN '2015/16'
		   WHEN replace(date,'-','') between '20160701' and '20170701' THEN '2016/17'
		   WHEN replace(date,'-','') between '20170701' and '20180701' THEN '2017/18'
		   WHEN replace(date,'-','') between '20180701' and '20190701' THEN '2018/19'
		   WHEN replace(date,'-','') between '20190701' and '20200701' THEN '2019/20'
		   WHEN replace(date,'-','') between '20200701' and '20210701' THEN '2020/21'
		   WHEN replace(date,'-','') between '20210701' and '20220701' THEN '2021/22'
		   WHEN replace(date,'-','') between '20220701' and '20230701' THEN '2022/23'
		   ELSE 'undefined'
		END as season
		,* from matches) m on (t.id=m.away_team_id or t.id=m.home_team_id)
		where league not like '%Reserve%'
		group by season,league,t.name 
order by season,league,points desc) m 
"""

df=query_database(query_teams_end_of_season,database="verbands.db")
df["rank"]=df.groupby(["season","league"])["points"].rank(method="dense", ascending=False)

df.head()

,name,league,season,points,url,rank
0,Oberwart,BVZ Burgenlandliga,2011/12,69,https://www.oefb.at/bewerbe/Spiel/Aufstellung/...,1.0
1,Neuberg,BVZ Burgenlandliga,2011/12,60,https://www.oefb.at/bewerbe/Spiel/Aufstellung/...,2.0
2,Marz,BVZ Burgenlandliga,2011/12,55,https://www.oefb.at/bewerbe/Spiel/Aufstellung/...,3.0
3,Draßburg,BVZ Burgenlandliga,2011/12,53,https://www.oefb.at/bewerbe/Spiel/Aufstellung/...,4.0
4,Baumgarten,BVZ Burgenlandliga,2011/12,53,https://www.oefb.at/bewerbe/Spiel/Aufstellung/...,4.0


In [12]:
#!pip install dtale
import dtale

d = dtale.show(df, notebook=True)

In [13]:
#Create a DAG from promotions and relegations
from graphlib import TopologicalSorter, CycleError

graph = {}

def promotions(data,graph):
    for index,row in data[data["rank"]==1].iterrows():
        if row["season"]!="undefined":
            new_season="/".join([str(int(row["season"].split("/")[0])+1),str(int(row["season"].split("/")[1])+1)])
            #new_season=row["season"].str.split("/")
            new_league=data.query('name == @row["name"] and season ==@new_season')["league"]
            #new_league=data[(data["name"]==row["name"])&(data["season"]==new_season)]["league"]
            if len(new_league)>0:
                if new_league.iloc[0]!=row["league"]:
                    if new_league.iloc[0] not in graph:
                        graph[new_league.iloc[0]]={row["league"]}
                    else:
                        graph[new_league.iloc[0]]=graph[new_league.iloc[0]]|{row["league"]}
    return graph

def relegations(data,graph):
    for index,row in data[data.groupby("league")['rank'].transform(max) == df['rank']].iterrows():
        if row["season"]!="undefined":
            new_season="/".join([str(int(row["season"].split("/")[0])+1),str(int(row["season"].split("/")[1])+1)])
            #new_season=row["season"].str.split("/")
            new_league=data.query('name == @row["name"] and season ==@new_season')["league"]
            #new_league=data[(data["name"]==row["name"])&(data["season"]==new_season)]["league"]
            if len(new_league)>0:
                if new_league.iloc[0]!=row["league"]:
                    if row["league"] not in graph:
                        graph[row["league"]]={new_league.iloc[0]}
                    else:
                        graph[row["league"]]=graph[row["league"]]|{new_league.iloc[0]}
    return graph

graph.update(promotions(df,graph))
graph.update(relegations(df,graph))
print(graph)

import networkx as nx

#edges = [('A', 'B'),('C', 'D'),('D', 'C'),('C', 'D')]

G = nx.DiGraph(graph)

for cycle in nx.simple_cycles(G):
    print(cycle)
    del graph[cycle[1]]

ts = TopologicalSorter(graph)
tuple(ts.static_order())


{'Regionalliga Ost': {'Burgenlandliga', 'BVZ Burgenlandliga'}, 'BVZ Burgenlandliga': {'II. Liga Süd'}, '1. Klasse Mitte': {'2. Klasse Mitte'}, 'II. Liga Mitte': {'1. Klasse Mitte'}, 'II. Liga Süd': {'1. Klasse Süd'}, '2. Kl. Mitte Cup  - 2 Kl. Cup B': {'2. Klasse Mitte'}, 'Burgenlandliga': {'II. Liga Süd'}, 'Planet Pure Frauen Bundesliga': {'Regionalliga Ost'}, 'tt.com RL Tirol - GD': {'Regionalliga West'}, 'Regionalliga West': {'tt.com RL Tirol - GD'}}
['tt.com RL Tirol - GD', 'Regionalliga West']


('2. Klasse Mitte',
 '1. Klasse Süd',
 'Regionalliga West',
 '1. Klasse Mitte',
 '2. Kl. Mitte Cup  - 2 Kl. Cup B',
 'II. Liga Süd',
 'tt.com RL Tirol - GD',
 'II. Liga Mitte',
 'BVZ Burgenlandliga',
 'Burgenlandliga',
 'Regionalliga Ost',
 'Planet Pure Frauen Bundesliga')

In [14]:
print(query_database("Select league, count(distinct(url)) from matches group by league"))

                                             league  count(distinct(url))
0                   1. Kl. Mitte Cup - 1. Kl. Cup A                     1
1                   1. Kl. Mitte Cup - 1. Kl. Cup D                     1
2                                   1. Klasse Mitte                   444
3                           1. Klasse Mitte Reserve                   424
4                                    1. Klasse Nord                   548
5                            1. Klasse Nord Reserve                   455
6                                     1. Klasse Süd                   251
7                                   1. Klasse Süd B                   156
8                             1. Klasse Süd Reserve                   142
9                   2. Kl. Mitte Cup  - 2 Kl. Cup B                     1
10                 2. Kl. Mitte Cup  - 2. Kl. Cup A                     1
11                 2. Kl. Mitte Cup  - 2. Kl. Cup C                     1
12                 2. Kl. Mitte Cup  -

Aktuell stehen aus 13 Ligen bis zu 13 Saisonen an Spielinformationen zur Verfügung. Die daraus gewonnen Daten werden als Basis für die Analyse verwendet.

In [15]:
df=query_database(query_spiele_spieler.format(player["name"].str.strip()),database="verbands.db") 
print(len(df))
print(df.columns)
print(df.head(1))

40
Index(['homeresult', 'awayresult', 'current_round', 'season', 'id', 'url',
       'date', 'starttime', 'league', 'round', 'result', 'away_team_id',
       'away_team_trainer_id', 'home_team_id', 'home_team_trainer_id', 'id',
       'player_id', 'match_id', 'team_id', 'position', 'goals', 'yellow_cards',
       'red_cards', 'starting_minute', 'id', 'url', 'name', 'nationality',
       'birthyear'],
      dtype='object')
  homeresult awayresult  current_round   season   id  \
0          0          5              6  2022/23  513   

                                                 url        date starttime  \
0  https://www.oefb.at/bewerbe/Spiel/Aufstellung/...  2022-09-16     18:00   

           league     round  ...    position  goals  yellow_cards  red_cards  \
0  2. Klasse Nord  6. Runde  ...  Mittelfeld      1             0          0   

   starting_minute    id                                                url  \
0             None  3800  https://www.oefb.at/Profile/Spieler/65

Für den Spieler haben wir aktuell 137 Spiele gefunden auf dessen Basis jetzt die einzelnen Statistiken berechnet werden.

## Analyse

### Spieler Statistik

| Saison | Team | Games | Wins | Draws | Looses | Scored | Conceded | Points/Game | Starting | In | Out | Goals | Yellow Cards | Red Cards |
|:-------- | -------- | -------- | -------- | -------- | -------- | -------- | -------- | -------- | -------- | -------- | -------- | -------- | -------- | -------- |
| 2022 | | | | | | | | | | | | | |

In [16]:
df["tore_geschossen_team"]=df.apply(lambda x: x["homeresult"] if x["home_team_id"]==x["team_id"] else x["awayresult"] ,axis=1).astype(int)
df["tore_bekommen_team"]=df.apply(lambda x: x["homeresult"] if x["away_team_id"]==x["team_id"] else x["awayresult"] ,axis=1).astype(int)

In [17]:
df.groupby(["season"]).agg({'tore_geschossen_team':['sum'],'tore_bekommen_team':['sum'],'goals':["sum"],'yellow_cards':['sum'],'red_cards':['sum']}).reset_index().style.hide_index()\
                .bar(subset=["tore_geschossen_team",], color='darkgreen')\
                .bar(subset=["tore_bekommen_team",], color='darkgreen')\
                .bar(subset=["goals",], color='darkblue')\
                .bar(subset=["yellow_cards",], color='darkblue')\
                .bar(subset=["red_cards",], color='darkblue')

C:\Users\marst\AppData\Local\Temp\ipykernel_26244\1216417428.py:1: FutureWarning:

this method is deprecated in favour of `Styler.hide(axis="index")`

c:\Users\marst\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\formats\style.py:4151: RuntimeWarning:

invalid value encountered in longlong_scalars



season,tore_geschossen_team,tore_bekommen_team,goals,yellow_cards,red_cards
,sum,sum,sum,sum,sum
2014/15,6,9,0,2,0
2018/19,61,26,4,4,0
2022/23,26,15,6,4,0


In [18]:

query_points_per_league_season_round_team="""
select scored-conceded as difference, *
from (
select 
sum(
	CASE WHEN homeresult==awayresult THEN 1 WHEN homeresult>awayresult and m.home_team_id==t.id THEN 3 WHEN homeresult<awayresult and m.away_team_id==t.id THEN 3 ELSE 0 END
	) OVER (
		PARTITION BY season,league,name
        ORDER BY current_round
    ) running_points,
	sum(
	CASE WHEN m.home_team_id==t.id THEN homeresult WHEN  m.away_team_id==t.id THEN awayresult ELSE 0 END
	) OVER (
		PARTITION BY season,league,name
        ORDER BY current_round
    ) scored,
    sum(
	CASE WHEN m.home_team_id==t.id THEN awayresult WHEN  m.away_team_id==t.id THEN homeresult  ELSE 0 END
	) OVER (
		PARTITION BY season,league,name
        ORDER BY current_round
    ) conceded,
	 *
from teams t join  (
select substr(result,0,instr(result, ':')) as homeresult,
		substr(result,instr(result, ':') + 1) as awayresult, cast(substr(round,0,instr(round, '.')) as INT) as current_round,
		CASE 
		   WHEN replace(date,'-','') between '20110701' and '20120701' THEN '2011/12'
		   WHEN replace(date,'-','') between '20120701' and '20130701' THEN '2012/13'
		   WHEN replace(date,'-','') between '20130701' and '20140701' THEN '2013/14'
		   WHEN replace(date,'-','') between '20140701' and '20150701' THEN '2014/15'
		   WHEN replace(date,'-','') between '20150701' and '20160701' THEN '2015/16'
		   WHEN replace(date,'-','') between '20160701' and '20170701' THEN '2016/17'
		   WHEN replace(date,'-','') between '20170701' and '20180701' THEN '2017/18'
		   WHEN replace(date,'-','') between '20180701' and '20190701' THEN '2018/19'
		   WHEN replace(date,'-','') between '20190701' and '20200701' THEN '2019/20'
		   WHEN replace(date,'-','') between '20200701' and '20210701' THEN '2020/21'
		   WHEN replace(date,'-','') between '20210701' and '20220701' THEN '2021/22'
		   WHEN replace(date,'-','') between '20220701' and '20230701' THEN '2022/23'
		   ELSE 'undefined'
		END as season
		,* from matches
		where current_round != 0 ) m on (t.id=m.away_team_id or t.id=m.home_team_id)
order by season,league,current_round asc,running_points desc) m where season='2021/22' and league="Regionalliga Ost"
"""

df=query_database(query_points_per_league_season_round_team,database="verbands.db")
df.head(10)

,difference,running_points,scored,conceded,id,name,year,homeresult,awayresult,current_round,...,url,date,starttime,league,round,result,away_team_id,away_team_trainer_id,home_team_id,home_team_trainer_id
0,2,3,3,1,267,ASK-BSC Bruck / Leitha,None,3,1,1,...,https://www.oefb.at/bewerbe/Spiel/Aufstellung/...,2021-07-30,17:00,Regionalliga Ost,1. Runde,3:1,196,300,267,308
1,2,3,5,3,197,FCM Traiskirchen,None,5,3,1,...,https://www.oefb.at/bewerbe/Spiel/Aufstellung/...,2021-07-30,17:30,Regionalliga Ost,1. Runde,5:3,200,312,197,309
2,1,3,1,0,118,TWL Elektra,None,1,0,1,...,https://www.oefb.at/bewerbe/Spiel/Aufstellung/...,2021-07-31,14:00,Regionalliga Ost,1. Runde,1:0,199,306,118,144
3,1,3,1,0,195,Wiener Sport-Club,None,1,0,1,...,https://www.oefb.at/bewerbe/Spiel/Aufstellung/...,2021-07-30,17:30,Regionalliga Ost,1. Runde,1:0,266,33,195,301
4,3,3,4,1,29,Wiener Viktoria,None,4,1,1,...,https://www.oefb.at/bewerbe/Spiel/Aufstellung/...,2021-07-31,16:00,Regionalliga Ost,1. Runde,4:1,202,313,29,34
5,0,1,2,2,24,Draßburg,None,2,2,1,...,https://www.oefb.at/bewerbe/Spiel/Aufstellung/...,2021-07-30,17:30,Regionalliga Ost,1. Runde,2:2,54,250,24,146
6,0,1,3,3,53,First Vienna FC 1894,None,3,3,1,...,https://www.oefb.at/bewerbe/Spiel/Aufstellung/...,2021-07-30,16:30,Regionalliga Ost,1. Runde,3:3,53,66,119,303
7,0,1,2,2,54,SC Neusiedl am See 1919,None,2,2,1,...,https://www.oefb.at/bewerbe/Spiel/Aufstellung/...,2021-07-30,17:30,Regionalliga Ost,1. Runde,2:2,54,250,24,146
8,0,1,3,3,119,Stripfing,None,3,3,1,...,https://www.oefb.at/bewerbe/Spiel/Aufstellung/...,2021-07-30,16:30,Regionalliga Ost,1. Runde,3:3,53,66,119,303
9,-1,0,0,1,199,Admira Juniors,None,1,0,1,...,https://www.oefb.at/bewerbe/Spiel/Aufstellung/...,2021-07-31,14:00,Regionalliga Ost,1. Runde,1:0,199,306,118,144


In [19]:
fig=px.scatter(df, x="running_points", y="difference", animation_frame="current_round", animation_group="name",
           color="name", hover_name="name", size_max=55)
fig.update_xaxes(range=[0, 60])
fig.update_yaxes(range=[-50, 100])
fig.show()
#fig.write_html("./plots/interactive_plotly.html")

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
df=query_database(query_points_per_league_season_round_team,database="verbands.db")

fig=px.line(df, x="current_round", y="running_points",
           color="name", hover_name="name")
#fig.update_xaxes(range=[0, 60])
#fig.update_yaxes(range=[-50, 100])
fig.show()
#fig.write_html("./plots/interactive_plotly.html")

In [ ]:
%%html
<style>
table {float:center}
</style>

## Einfach Statistiken


### Trainer bei Draßburg Saison 2021/22

|Statistiken: | Trainer Mandl | Anderer Trainer 
|:-------- | -------- | -------- 
|Durchschnittliche Punkte pro Spiel | 0.8 |  0.571
|Durchschnittliche Tore pro Spiel Geschossen/Bekommen | 1.2 / 1.8 | 0.76 / 2.19
|Durchschnittliche Karten pro Spiel | 1.6 / 0.2 |  2.24 / 0.1
|Anzahl verwendeter Spieler| 23 | 35


In [ ]:
# GET TABLE
query_table_averag_points_mandl=("""
Select m.league,t.name, 
sum(
CASE WHEN homeresult==awayresult THEN 1 WHEN homeresult>awayresult and m.home_team_id==t.id THEN 3 WHEN homeresult<awayresult and m.away_team_id==t.id THEN 3 ELSE 0 END
) as 'Points' ,
count(*) as 'Games', 
avg(
CASE WHEN homeresult==awayresult THEN 1 WHEN homeresult>awayresult and m.home_team_id==t.id THEN 3 WHEN homeresult<awayresult and m.away_team_id==t.id THEN 3 ELSE 0 END
) as 'Average Points',
sum(
	CASE WHEN m.home_team_id==t.id THEN homeresult WHEN  m.away_team_id==t.id THEN awayresult ELSE 0 END
	) as scored,
sum(
	CASE WHEN m.home_team_id==t.id THEN awayresult WHEN  m.away_team_id==t.id THEN homeresult  ELSE 0 END
	) as conceded
from teams t join 
	(select substr(result,0,instr(result, ':')) as homeresult,
	substr(result,instr(result, ':') + 1) as awayresult,cast(substr(round,0,instr(round, '.')) as INT) as current_round,* from matches) m 
on (t.id=m.away_team_id or t.id=m.home_team_id) 
--join trainers c on (m.home_team_trainer_id=c.id or m.away_team_trainer_id=m.id)
where replace(date,'-','') between '{}' and '{}' 
         and league="Regionalliga Ost"
         -- and ((t.name like '%{}%' and t.id=m.home_team_id and c.id=m.home_team_trainer_id) or (t.name like '%{}%' and t.id=m.away_team_id and c.id=m.away_team_trainer_id)) 
group by m.league,t.name 
order by points desc""")

In [ ]:
df=query_database(query_table_averag_points_mandl.format(trainer_zeit[0],trainer_zeit[1],trainer_zeit[0],trainer_zeit[1],trainer_zeit[0],trainer_zeit[1],vereine[0],vereine[0]),database="verbands.db")
df["Difference"]=df["scored"]-df["conceded"]
df.head(50).style.hide_index()\
                 .bar(subset=["Points",], color='lightgreen')\
                 .bar(subset=["Average Points"], color='#FFA07A')\
                .bar(subset=["Difference"], color='lightblue')

NameError: name 'trainer_zeit' is not defined

In [ ]:
# GET TABLE
query_table=("""
Select m.league,t.name, 
sum(
CASE WHEN homeresult==awayresult THEN 1 WHEN homeresult>awayresult and m.home_team_id==t.id THEN 3 WHEN homeresult<awayresult and m.away_team_id==t.id THEN 3 ELSE 0 END
) as 'Points' ,
count(*) as 'Games', 
avg(
CASE WHEN homeresult==awayresult THEN 1 WHEN homeresult>awayresult and m.home_team_id==t.id THEN 3 WHEN homeresult<awayresult and m.away_team_id==t.id THEN 3 ELSE 0 END
) as 'Average Points',
sum(
	CASE WHEN m.home_team_id==t.id THEN homeresult WHEN  m.away_team_id==t.id THEN awayresult ELSE 0 END
	) as scored,
sum(
	CASE WHEN m.home_team_id==t.id THEN awayresult WHEN  m.away_team_id==t.id THEN homeresult  ELSE 0 END
	) as conceded
from teams t join 
	(select substr(result,0,instr(result, ':')) as homeresult,
	substr(result,instr(result, ':') + 1) as awayresult,cast(substr(round,0,instr(round, '.')) as INT) as current_round,* from matches) m 
on (t.id=m.away_team_id or t.id=m.home_team_id) 
--join trainers c on (m.home_team_trainer_id=c.id or m.away_team_trainer_id=m.id)
where replace(date,'-','') between '{}' and '{}' 
         and league="Regionalliga Ost"
         -- and ((t.name like '%{}%' and t.id=m.home_team_id and c.id=m.home_team_trainer_id) or (t.name like '%{}%' and t.id=m.away_team_id and c.id=m.away_team_trainer_id)) 

group by m.league,t.name 
order by points desc""")

df=query_database(query_table.format("20210701","20220426",vereine[0],vereine[0]),database="verbands.db")
df["Difference"]=df["scored"]-df["conceded"]
df.head(50).style.hide_index()\
                 .bar(subset=["Points",], color='lightgreen')\
                 .bar(subset=["Average Points",], color='#FFA07A')\
                .bar(subset=["Difference",], color='lightblue')


league,name,Points,Games,Average Points,scored,conceded,Difference
Regionalliga Ost,First Vienna FC 1894,47,21,2.238095,43,15,28
Regionalliga Ost,Stripfing,42,21,2.000000,47,15,32
Regionalliga Ost,SV Leobendorf,38,21,1.809524,38,28,10
Regionalliga Ost,TWL Elektra,37,21,1.761905,29,24,5
Regionalliga Ost,Wiener Sport-Club,35,21,1.666667,40,27,13
Regionalliga Ost,FC Marchfeld / Mannsdorf-Große.,33,21,1.571429,36,31,5
Regionalliga Ost,FCM Traiskirchen,32,20,1.600000,36,31,5
Regionalliga Ost,SC Neusiedl am See 1919,30,20,1.500000,29,30,-1
Regionalliga Ost,Wiener Viktoria,28,21,1.333333,31,32,-1
Regionalliga Ost,ASK-BSC Bruck/Leitha,22,21,1.047619,36,35,1


In [ ]:
query_table_cards_gotten=("""
Select m.league,t.name, 
sum(
CASE WHEN homeresult==awayresult THEN 1 WHEN homeresult>awayresult and m.home_team_id==t.id THEN 3 WHEN homeresult<awayresult and m.away_team_id==t.id THEN 3 ELSE 0 END
) as 'Points' ,
count(*) as 'Games', 
avg(
CASE WHEN homeresult==awayresult THEN 1 WHEN homeresult>awayresult and m.home_team_id==t.id THEN 3 WHEN homeresult<awayresult and m.away_team_id==t.id THEN 3 ELSE 0 END
) as 'Average Points',
sum(
	CASE WHEN m.home_team_id==t.id THEN homeresult WHEN  m.away_team_id==t.id THEN awayresult ELSE 0 END
	) as scored,
sum(
	CASE WHEN m.home_team_id==t.id THEN awayresult WHEN  m.away_team_id==t.id THEN homeresult  ELSE 0 END
	) as conceded,
sum(	
	CASE WHEN m.home_team_id==t.id THEN p.yellow_cards  ELSE 0 END
) as yellow_cards_gotten,
sum(	
	CASE WHEN m.home_team_id==t.id THEN p.red_cards  ELSE 0 END
) as red_cards_gotten
from teams t join 
	(select substr(result,0,instr(result, ':')) as homeresult,
	substr(result,instr(result, ':') + 1) as awayresult,cast(substr(round,0,instr(round, '.')) as INT) as current_round,* from matches) m 
on (t.id=m.away_team_id or t.id=m.home_team_id) join 
(select m.id as id, sum(p.yellow_cards) as yellow_cards, sum(p.red_cards) as red_cards 
from (select distinct player_id,match_id,red_cards,yellow_cards from matches_players) p join matches m on p.match_id=m.id 
where (p.yellow_cards>0 or p.red_cards>0) and league="Regionalliga Ost" 
group by m.id) p on p.id=m.id
where replace(date,'-','') between '{}' and '{}'
         and league="Regionalliga Ost"
group by m.league,t.name
order by "Points" DESC""")

In [ ]:
df=query_database(query_table_cards_gotten.format(trainer_zeit[0],trainer_zeit[1]),database="verbands.db")
df["Difference"]=df["scored"]-df["conceded"]
df.head(50).style.hide_index()\
                 .bar(subset=["Points",], color='lightgreen')\
                 .bar(subset=["yellow_cards_gotten"], color='#FFA07A')\
                .bar(subset=["red_cards_gotten"], color='lightblue')


league,name,Points,Games,Average Points,scored,conceded,yellow_cards_gotten,red_cards_gotten,Difference
Regionalliga Ost,Stripfing,12,5,2.400000,13,4,6,0,9
Regionalliga Ost,First Vienna FC 1894,11,5,2.200000,9,3,12,0,6
Regionalliga Ost,SC Neusiedl am See 1919,11,6,1.833333,10,7,12,1,3
Regionalliga Ost,Wiener Sport-Club,11,5,2.200000,8,5,8,0,3
Regionalliga Ost,SC Wiener Neustadt,10,5,2.000000,12,6,17,1,6
Regionalliga Ost,Wiener Viktoria,10,5,2.000000,5,4,5,0,1
Regionalliga Ost,FC Marchfeld / Mannsdorf-Große.,7,5,1.400000,8,10,13,0,-2
Regionalliga Ost,SV Leobendorf,7,5,1.400000,7,9,14,0,-2
Regionalliga Ost,ASK-BSC Bruck/Leitha,5,5,1.000000,7,9,14,0,-2
Regionalliga Ost,Draßburg,4,5,0.800000,6,9,8,1,-3


In [ ]:
df=query_database(query_table_cards_gotten.format("20210701","20220426",vereine[0],vereine[0]),database="verbands.db")
df["Difference"]=df["scored"]-df["conceded"]
df.head(50).style.hide_index()\
                 .bar(subset=["Points",], color='lightgreen')\
                 .bar(subset=["yellow_cards_gotten"], color='#FFA07A')\
                .bar(subset=["red_cards_gotten"], color='lightblue')


league,name,Points,Games,Average Points,scored,conceded,yellow_cards_gotten,red_cards_gotten,Difference
Regionalliga Ost,First Vienna FC 1894,47,21,2.238095,43,15,39,1,28
Regionalliga Ost,Stripfing,42,21,2.000000,47,15,41,2,32
Regionalliga Ost,SV Leobendorf,38,21,1.809524,38,28,48,0,10
Regionalliga Ost,TWL Elektra,37,21,1.761905,29,24,41,0,5
Regionalliga Ost,Wiener Sport-Club,35,21,1.666667,40,27,52,2,13
Regionalliga Ost,FCM Traiskirchen,32,20,1.600000,36,31,37,1,5
Regionalliga Ost,FC Marchfeld / Mannsdorf-Große.,30,20,1.500000,34,31,40,1,3
Regionalliga Ost,SC Neusiedl am See 1919,30,19,1.578947,29,26,38,0,3
Regionalliga Ost,Wiener Viktoria,25,20,1.250000,27,32,48,1,-5
Regionalliga Ost,ASK-BSC Bruck/Leitha,22,21,1.047619,36,35,54,1,1


In [ ]:
# GET TABLE
query_table=("""
select  m.league,t.name, count(DISTINCT(player_id)) as "Unterschiedliche_Spieler"
from 
	(select substr(result,0,instr(result, ':')) as homeresult,
	substr(result,instr(result, ':') + 1) as awayresult,
	cast(substr(round,0,instr(round, '.')) as INT) as current_round,* from matches) m 
join matches_players p on p.match_id=m.id join teams t on (t.id=p.team_id)
where replace(date,'-','') between '{}' and '{}' 
         and league="Regionalliga Ost"
group by m.league,t.name
order by Unterschiedliche_Spieler desc""")

In [ ]:
df=query_database(query_table.format(trainer_zeit[0],trainer_zeit[1],vereine[0],vereine[0]),database="verbands.db")
df.head(50).style.hide_index()\
                .bar(subset=["Unterschiedliche_Spieler"], color='lightblue')

league,name,Unterschiedliche_Spieler
Regionalliga Ost,Admira Juniors,30
Regionalliga Ost,FC Mauerwerk,24
Regionalliga Ost,SC Neusiedl am See 1919,24
Regionalliga Ost,Stripfing,24
Regionalliga Ost,Draßburg,23
Regionalliga Ost,TWL Elektra,23
Regionalliga Ost,SC Wiener Neustadt,21
Regionalliga Ost,Wiener Sport-Club,21
Regionalliga Ost,FCM Traiskirchen,20
Regionalliga Ost,First Vienna FC 1894,20


In [ ]:
df=query_database(query_table.format("20210701","20220426",vereine[0],vereine[0]),database="verbands.db")
df.head(50).style.hide_index()\
                .bar(subset=["Unterschiedliche_Spieler"], color='lightblue')

league,name,Unterschiedliche_Spieler
Regionalliga Ost,Admira Juniors,44
Regionalliga Ost,FC Mauerwerk,41
Regionalliga Ost,Draßburg,35
Regionalliga Ost,SC Wiener Neustadt,34
Regionalliga Ost,SC Neusiedl am See 1919,33
Regionalliga Ost,Stripfing,32
Regionalliga Ost,Wiener Viktoria,30
Regionalliga Ost,FCM Traiskirchen,29
Regionalliga Ost,FC Marchfeld / Mannsdorf-Große.,28
Regionalliga Ost,First Vienna FC 1894,27


## Fitness: Wann wurden Tor geschossen/bekommen

In [ ]:
# When Goals Scored (Fitness)
# GET TABLE
query_table_goals_scored=("""
select  m.league,t.name,minute, count(*) as "Goals_per_min"
from 
(select substr(result,0,instr(result, ':')) as homeresult,
	substr(result,instr(result, ':') + 1) as awayresult,
	cast(substr(round,0,instr(round, '.')) as INT) as current_round,* from matches) m 
join (select DISTINCT match_id,minute,type,team_id from matches_activity) p on p.match_id=m.id 
join teams t on (t.id=p.team_id)
where replace(date,'-','') between '{}' and '{}' 
         and league="Regionalliga Ost" and type="goal"
group by m.league,t.name, minute
order by m.league,t.name asc
""")


# GET TABLE
query_table_goals_conceeded=("""
select  m.league,t.name,minute, count(*) as "Goals_per_min_conceeded"
from 
(select substr(result,0,instr(result, ':')) as homeresult,
	substr(result,instr(result, ':') + 1) as awayresult,
	cast(substr(round,0,instr(round, '.')) as INT) as current_round,* from matches) m 
join   (select DISTINCT match_id,minute,type,team_id from matches_activity) a on a.match_id=m.id 
join teams t on (a.team_id<>t.id and (t.id=m.away_team_id or t.id=m.home_team_id))
where replace(date,'-','') between '{}' and '{}' 
         and league="Regionalliga Ost" and type="goal"
group by m.league,t.name, minute
order by t.name,minute
""")

In [ ]:
df=query_database(query_table_goals_scored.format(trainer_zeit[0],trainer_zeit[1],vereine[0],vereine[0]),database="verbands.db")
fig=px.histogram(df, x="minute",y='Goals_per_min' ,color="name",nbins=6,title="Wann Tore geschossen mit Mandl")
fig.update_layout(barmode='overlay')
fig.update_traces(xbins=dict( # bins used for histogram
        start=0.0,
        end=90.0,
        size=15
    ))
fig.show()

In [ ]:
df=query_database(query_table_goals_scored.format("20210701","20220426",vereine[0],vereine[0]),database="verbands.db")
fig=px.histogram(df, x="minute",y='Goals_per_min' ,color="name",nbins=6,title="Wann Tore geschossen vor Mandl")
fig.update_layout(barmode='overlay')
fig.update_traces(xbins=dict( # bins used for histogram
        start=0.0,
        end=90.0,
        size=15
    ))
fig.show()

In [ ]:
df=query_database(query_table_goals_conceeded.format(trainer_zeit[0],trainer_zeit[1],vereine[0],vereine[0]),database="verbands.db")
fig=px.histogram(df, x="minute",y='Goals_per_min_conceeded' ,color="name",title="Wann Tore bekommen mit Mandl")
fig.update_layout(barmode='overlay')
fig.update_traces(xbins=dict( # bins used for histogram
        start=0.0,
        end=90.0,
        size=10
    ))
fig.show()

In [ ]:
df=query_database(query_table_goals_conceeded.format("20210701","20220426",vereine[0],vereine[0]),database="verbands.db")
fig=px.histogram(df, x="minute",y='Goals_per_min_conceeded' ,color="name",title="Wann Tore bekommen vor Mandl")
fig.update_layout(barmode='overlay')
fig.update_traces(xbins=dict( # bins used for histogram
        start=0.0,
        end=90.0,
        size=10
    ))
fig.show()

## Wie wurden die Tore geschossen

In [ ]:
def goal_type(y:str):
    x=y.lower()
    #print(x)
    if "strafstoß" in x:
        return "strafstoß"
    elif "freistoß" in x:
        return "freistoß"
    elif "eigentor" in x:
        return "eigentor"
    return "in_game"
# GET TABLE
query_table=("""
select  *--count(*) as "Goals_per_min_scired"
from  (select DISTINCT match_id,minute,type,team_id,text from matches_activity) a 
join matches m on m.id=a.match_id
join  teams t on a.team_id==t.id 
where type="goal" and replace(date,'-','') between '{}' and '{}' 
         and league="Regionalliga Ost" and type="goal"
--group by m.league,t.name, minute
order by minute,name
""")

In [ ]:
df=query_database(query_table.format(trainer_zeit[0],trainer_zeit[1],vereine[0],vereine[0]),database="verbands.db")
df["goal_type"]=df["text"].apply(lambda x: goal_type(x))
fig=px.histogram(df, x="goal_type" ,color="name",title="Wie wurden Tore geschossen mit Mandl")
fig.update_layout(barmode='stack') #"stack", "relative", "group"
fig.show()

In [ ]:
df.groupby(["name","goal_type"])["goal_type"].count().reset_index(name='count').style.hide_index()

name,goal_type,count
ASK-BSC Bruck/Leitha,in_game,6
ASK-BSC Bruck/Leitha,strafstoß,1
Admira Juniors,in_game,7
Draßburg,in_game,6
FC Marchfeld / Mannsdorf-Große.,eigentor,1
FC Marchfeld / Mannsdorf-Große.,in_game,8
FC Mauerwerk,in_game,5
FCM Traiskirchen,in_game,3
FCM Traiskirchen,strafstoß,2
First Vienna FC 1894,in_game,9


In [ ]:
df=query_database(query_table.format("20210701","20220426",vereine[0],vereine[0]),database="verbands.db")
df["goal_type"]=df["text"].apply(lambda x: goal_type(x))
fig=px.histogram(df, x="goal_type" ,color="name",title="Wie wurden Tore geschossen vor Mandl")
fig.update_layout(barmode='stack') #"stack", "relative", "group"
fig.show()

In [ ]:
df.groupby(["name","goal_type"])["goal_type"].count().reset_index(name='count').style.hide_index()


name,goal_type,count
ASK-BSC Bruck/Leitha,freistoß,1
ASK-BSC Bruck/Leitha,in_game,33
ASK-BSC Bruck/Leitha,strafstoß,2
Admira Juniors,eigentor,1
Admira Juniors,in_game,25
Admira Juniors,strafstoß,1
Draßburg,in_game,15
FC Marchfeld / Mannsdorf-Große.,in_game,33
FC Marchfeld / Mannsdorf-Große.,strafstoß,2
FC Mauerwerk,in_game,22


In [ ]:
# GET TABLE
query_table=("""
select  *--count(*) as "Goals_per_min_conceeded"
from  (select DISTINCT match_id,minute,type,team_id,text from matches_activity) a 
join matches m on m.id=a.match_id
join  teams t on (t.id=m.away_team_id or t.id=m.home_team_id)
where type="goal" and replace(date,'-','') between '{}' and '{}' 
		and a.team_id<>t.id 
         and league="Regionalliga Ost" and type="goal"
--group by m.league,t.name, minute
order by minute,name
""")


In [ ]:
df=query_database(query_table.format(trainer_zeit[0],trainer_zeit[1],vereine[0],vereine[0]),database="verbands.db")
df["goal_type"]=df["text"].apply(lambda x: goal_type(x))
df.groupby(["name","goal_type"])["goal_type"].count()
df.head()
fig=px.histogram(df, x="goal_type" ,color="name",title="Wie wurden Tore bekommen mit Mandl")
fig.update_layout(barmode='stack') #"stack", "relative", "group"
fig.show()

In [ ]:
df.groupby(["name","goal_type"])["goal_type"].count()

name                             goal_type
ASK-BSC Bruck/Leitha             freistoß      1
                                 in_game       7
                                 strafstoß     1
Admira Juniors                   in_game      13
                                 strafstoß     1
Draßburg                         in_game       8
                                 strafstoß     1
FC Marchfeld / Mannsdorf-Große.  in_game       9
FC Mauerwerk                     in_game       7
                                 strafstoß     1
FCM Traiskirchen                 in_game      11
First Vienna FC 1894             in_game       3
SC Neusiedl am See 1919          in_game       7
SC Wiener Neustadt               eigentor      1
                                 in_game       5
                                 strafstoß     1
SV Leobendorf                    in_game       8
                                 strafstoß     1
Stripfing                        in_game       4
TWL Elektra               

In [ ]:
df=query_database(query_table.format("20210701","20220426",vereine[0],vereine[0]),database="verbands.db")
df["goal_type"]=df["text"].apply(lambda x: goal_type(x))
df.groupby(["name","goal_type"])["goal_type"].count()
df.head()
fig=px.histogram(df, x="goal_type" ,color="name",title="Wie wurden Tore bekommen vor Mandl")
fig.update_layout(barmode='stack') #"stack", "relative", "group"
fig.show()

In [ ]:
df.groupby(["name","goal_type"])["goal_type"].count()

name                             goal_type
ASK-BSC Bruck/Leitha             eigentor      1
                                 in_game      32
                                 strafstoß     2
Admira Juniors                   freistoß      1
                                 in_game      37
Draßburg                         in_game      45
                                 strafstoß     2
FC Marchfeld / Mannsdorf-Große.  in_game      30
                                 strafstoß     2
FC Mauerwerk                     eigentor      1
                                 in_game      45
                                 strafstoß     3
FCM Traiskirchen                 in_game      30
                                 strafstoß     1
First Vienna FC 1894             eigentor      2
                                 in_game      14
                                 strafstoß     1
SC Neusiedl am See 1919          in_game      29
SC Wiener Neustadt               freistoß      1
                          

## Trainer Einfluss
### Substitutions / Minute

In [ ]:
def convert_minute(x:str):
    if "HZ" in x:
        return 45
    if "+" in x:
        return int(eval(x[:-1]))
    else:
        return int(x[:-1])
    
def convert_sub_minute(x:str):
    return min([convert_minute(i) for i in x.split(";")])

# GET TABLE
query_table=("""
select *
from  (select DISTINCT match_id,minute, type,team_id,text from matches_activity) a 
join  matches m on m.id=a.match_id
join  teams t on (t.id=m.away_team_id or t.id=m.home_team_id)
where replace(date,'-','') between '20210701' and '20220426' 
		and a.team_id<>t.id 
         and league="Regionalliga Ost" and type="substitution"
--group by m.league,t.name, minute
order by minute,name
""")

In [ ]:
df=query_database(query_table.format(trainer_zeit[0],trainer_zeit[1],vereine[0],vereine[0]),database="verbands.db")
df["min"]=df["minute"].apply(lambda x: convert_minute(x))
fig=px.histogram(df, x="min" ,color="name",title="Auswechslungen in Minute mit Mandl")
fig.update_layout(barmode='overlay') #"stack", "relative", "group","overlay"
fig.show()

In [ ]:
df=query_database(query_table.format("20210701","20220426",vereine[0],vereine[0]),database="verbands.db")
df["min"]=df["minute"].apply(lambda x: convert_minute(x))
fig=px.histogram(df, x="min" ,color="name",title="Auswechslungen in Minute vor Mandl")
fig.update_layout(barmode='overlay') #"stack", "relative", "group","overlay"
fig.show()

### Analyse Wechselzeitpunkt auf Ergebnis

In [ ]:

# GET TABLE
query_table=("""
Select m.league,t.name, m.round, (CASE WHEN homeresult==awayresult THEN 1 WHEN homeresult>awayresult and m.home_team_id==t.id THEN 3 WHEN homeresult<awayresult and m.away_team_id==t.id THEN 3 ELSE 0 END ) as 'Points',*
from 
(select DISTINCT match_id,team_id,GROUP_CONCAT(minute,';') as "first_substitution_min", count(type) as "num_subs" from (select DISTINCT match_id,team_id,minute,type from matches_activity) where type = 'substitution' group by match_id,team_id) a 
join (select substr(result,0,instr(result, ':')) as homeresult, substr(result,instr(result, ':') + 1) as awayresult,cast(substr(round,0,instr(round, '.')) as INT) as current_round,* from matches) m 
on m.id=a.match_id 
join  teams t on (t.id=m.away_team_id or t.id=m.home_team_id)
where replace(date,'-','') between '20210701' and '20220426' 
         and league="Regionalliga Ost"
		 and a.team_id==t.id  
order by date,starttime
""")

df=query_database(query_table.format("20210701","20220426",vereine[0],vereine[0]),database="verbands.db")
df["min"]=df["first_substitution_min"].apply(lambda x: convert_sub_minute(x))

In [ ]:
from statsmodels.api import MNLogit
MNLogit(df["Points"],df.loc[:,['num_subs','min']]).fit().summary()

Optimization terminated successfully.
         Current function value: 1.075713
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                 Points   No. Observations:                  292
Model:                        MNLogit   Df Residuals:                      288
Method:                           MLE   Df Model:                            2
Date:                Tue, 04 Oct 2022   Pseudo R-squ.:                0.004684
Time:                        22:47:54   Log-Likelihood:                -314.11
converged:                       True   LL-Null:                       -315.59
Covariance Type:            nonrobust   LLR p-value:                    0.2280
==============================================================================
  Points=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
num_subs      -0.2531      0.133     -1.901      0.057      -0.514       0.008
min            0.0058      0.007      0.883      0.377      -0.007       0.019
------------------------------------------------------------------------------
  Points=3       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
num_subs      -0.1266      0.117     -1.082      0.279      -0.356       0.103
min            0.0078      0.006      1.316      0.188      -0.004       0.019
==============================================================================
"""

#### Result

Einwechselzeitpunkt hat generell keine Auswirkungen auf das Ergebnis

### Kontakt

Email: mar.stagl@gmail.com

Tel: 0660 5755814
